<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


## Adversarial Search (25 Points)

HW3 :: Practical Question 1

<br>

Corresponding TA: Aryan Ahadinia, Saeed Hedayatiyan


Consider the given $8\times8$ board game. There are two players in this game and they try to block each other and limit their valid moves by making walls. Their moves are limited to the 8 cells (or less when they are around the edges) beside them and the walls will start to appear only when a player steps into a random generated yellow cell and the wall will continue to be built until the player reaches the next yellow cell. The players start the game at $(0, 0)$ and $(7, 7)$ and they keep playing until one of them loses. There are multiple ways for a player to win the match:

- If a player steps into a wall or out of the board, the player loses the match!
- If the players reach the maximum moves the game finishes and the winner would be chosen based on the maximum valid move they have left plus the number of walls they have made and "No more moves" massege will appear on terminal.
- If the getMove() function does not return any value after 2 seconds, the player loses the match and "Time limit exceeded" massage will appear on terminal.

<center>
<img src="assets/Q3.png" style="width: 50%"/>
</center>


In this question, the codes for starting the game were provided for you in Helper-codes folder. You are expected to implement the MinimaxPlayer class (which inherit the Player class) with alpha-beta pruning.

In [1]:
%pip install tk


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: C:\Users\mjmah\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import random
import time

from Helper_codes.graphics import *
from Helper_codes.question3 import Cell
from Helper_codes.question3 import IntPair
from Helper_codes.question3 import Player
from Helper_codes.question3 import NaivePlayer
from Helper_codes.question3 import Board
from Helper_codes.question3 import Game


You can run cell below and watch two naive players.

In [2]:
p1 = NaivePlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

numberOfMatches = 1
score1, score2 = g.start(numberOfMatches)

print(score1 / numberOfMatches)


Player 2 has exceeded the time limit
Player 1 has won

1.0


In [8]:
import numpy as np
import random
from copy import deepcopy

# from timeit import default_timer as timer


class MinimaxPlayer(Player):
    time_ = 0
    g = None

    def __init__(self, col, x, y, max_depth=4):
        super().__init__(col, x, y)
        self.max_depth = max_depth

    # def my_move(self, board, nextPlace, playerColor):
    #     board.setNumberOfMoves(board.getNumberOfMoves() + 1)
    #
    #     board.getCell(nextPlace.x, nextPlace.y).setColor(playerColor)
    #
    #     board.players[playerColor-1].setX(nextPlace.x)
    #     board.players[playerColor-1].setY(nextPlace.y)

    def calculate_value(self, board):
        xx = abs(board.getPlayerX(1) - board.getPlayerX(2))
        yy = abs(board.getPlayerY(1) - board.getPlayerY(2))
        if xx == 0 and yy == 0:
            return board.getScore(1) - board.getScore(2) + 4
        elif xx == 0:
            return board.getScore(1) - board.getScore(2) + 2 + 1 / yy
        elif yy == 0:
            return board.getScore(1) - board.getScore(2) + 2 + 1 / xx
        return board.getScore(1) - board.getScore(2) + 1/xx + 1/yy

    def check_if_can_move(self, board, nextPlace, currentPlace, playerColor):
        if ((nextPlace.x < 0) | (nextPlace.y < 0) | (nextPlace.x >= 8) | (nextPlace.y >= 8)):
            return False
        elif ((abs(currentPlace.y - nextPlace.y) > 1) | (abs(currentPlace.x - nextPlace.x) > 1)):
            print("Invalid Call!")
            return False
        elif (board.getCell(nextPlace.x, nextPlace.y).getColor() != 0):
            return False
        elif ((nextPlace.x == board.players[2 - playerColor].getX()) and (
                nextPlace.y == board.players[2 - playerColor].getY())):
            return False
        elif (board.getNumberOfMoves() + 1 > 80):
            return False
        return True

    def min_value(self, board, alpha, beta, depth):
        # print("MinValue - Depth = " + str(depth))
        value = 1000000
        if depth == self.max_depth:
            return self.calculate_value(board), None
        min_action = -1
        array = [0, 1, 2, 3, 4, 5, 6, 7]
        from random import shuffle
        shuffle(array)
        for counter in array:
            pre_x = board.getPlayerX(0)
            pre_y = board.getPlayerY(0)
            currentPlace = IntPair(pre_x, pre_y)
            if counter == 0:
                self.setX(pre_x + 1)
                self.setY(pre_y + 1)
            elif counter == 1:
                self.setX(pre_x + 1)
                self.setY(pre_y + 0)
            elif counter == 2:
                self.setX(pre_x + 1)
                self.setY(pre_y - 1)
            elif counter == 3:
                self.setX(pre_x + 0)
                self.setY(pre_y + 1)
            elif counter == 4:
                self.setX(pre_x + 0)
                self.setY(pre_y - 1)
            elif counter == 5:
                self.setX(pre_x - 1)
                self.setY(pre_y + 1)
            elif counter == 6:
                self.setX(pre_x - 1)
                self.setY(pre_y + 0)
            elif counter == 7:
                self.setX(pre_x - 1)
                self.setY(pre_y - 1)
            x = self.getX()
            y = self.getY()
            nxt = IntPair(x, y)
            if self.check_if_can_move(board, nxt, currentPlace, 2) == False:
                continue
            copied_board = deepcopy(board)

            move_ = copied_board.move(nxt, 2)
            answer = [0, 0]
            if move_ == -1:
                answer[0] = 1000
            else:
                if copied_board.getNumberOfMoves() == copied_board.maxNumberOfMoves:
                    answer[0] = self.calculate_value(copied_board)
                else:
                    answer = self.max_value(Board(copied_board), alpha, beta, depth + 1)
            self.setX(pre_x)
            self.setY(pre_y)
            if answer[0] < alpha:
                return answer[0], counter
            if answer[0] < value:
                value = answer[0]
                min_action = counter
            beta = min(beta, value)

        if value == 1000000:
            # return 20000 ,0
            return self.calculate_value(board), -8
        return value, min_action

    def max_value(self, board, alpha, beta, depth):
        # print("MaxValue - Depth = " + str(depth))
        if depth == self.max_depth:
            return self.calculate_value(board), None

        value = -1000000
        max_action = -1
        array = [0, 1, 2, 3, 4, 5, 6, 7]
        from random import shuffle
        shuffle(array)
        for counter in array:
            pre_x = board.getPlayerX(1)
            pre_y = board.getPlayerY(1)
            currentPlace = IntPair(pre_x, pre_y)
            if counter == 0:
                self.setX(pre_x + 1)
                self.setY(pre_y + 1)
            elif counter == 1:
                self.setX(pre_x + 1)
                self.setY(pre_y + 0)
            elif counter == 2:
                self.setX(pre_x + 1)
                self.setY(pre_y - 1)
            elif counter == 3:
                self.setX(pre_x + 0)
                self.setY(pre_y + 1)
            elif counter == 4:
                self.setX(pre_x + 0)
                self.setY(pre_y - 1)
            elif counter == 5:
                self.setX(pre_x - 1)
                self.setY(pre_y + 1)
            elif counter == 6:
                self.setX(pre_x - 1)
                self.setY(pre_y + 0)
            elif counter == 7:
                self.setX(pre_x - 1)
                self.setY(pre_y - 1)
            x = self.getX()
            y = self.getY()
            nxt = IntPair(x, y)
            if self.check_if_can_move(board, nxt, currentPlace, 1) == False:
                continue
            copied_board = deepcopy(board)
            move_ = copied_board.move(nxt, 1)
            answer = [0, 0]
            if move_ == -1:
                answer[0] = -100000
            else:
                if copied_board.getNumberOfMoves() == copied_board.maxNumberOfMoves:
                    answer[0] = self.calculate_value(copied_board)
                else:
                    answer = self.min_value(Board(copied_board), alpha, beta, depth + 1)
            self.setX(pre_x)
            self.setY(pre_y)
            if answer[0] > beta:
                return answer[0], counter
            if answer[0] > value:
                value = answer[0]
                max_action = counter
            alpha = max(alpha, value)
        if value == -1000000:
            if depth == 0:
                print("WHAT?")
                return -10, -10
            return self.calculate_value(board), -8
        return value, max_action

    def getMove(self, board):
        x_next = board.getPlayerX(1)
        y_next = board.getPlayerY(1)
        self.setX(x_next)
        self.setY(y_next)
        x_next_opponent = board.getPlayerX(2)
        y_next_opponent = board.getPlayerY(2)

        pre_x = x_next
        pre_y = y_next

        max_ = self.max_value(deepcopy(board), -100000, 100000, 0)

        if max_[0] == -10:
            return IntPair(-10, -10)
        counter = max_[1]

        if counter == 0:
            x_next = x_next + 1
            y_next = y_next + 1

        elif counter == 1:
            x_next = x_next + 1
            y_next = y_next + 0

        elif counter == 2:
            x_next = x_next + 1
            y_next = y_next - 1

        elif counter == 3:
            x_next = x_next + 0
            y_next = y_next + 1


        elif counter == 4:
            x_next = x_next + 0
            y_next = y_next - 1

        elif counter == 5:
            x_next = x_next - 1
            y_next = y_next + 1

        elif counter == 6:
            x_next = x_next - 1
            y_next = y_next + 0

        elif counter == 7:
            x_next = x_next - 1
            y_next = y_next - 1
        # board.players[0].setX(pre_x)
        # board.players[0].setY(pre_y)
        # Game.getPlayers()[0].setX(pre_x)
        # Game.getPlayers()[0].setY(pre_y)
        g1 = MinimaxPlayer.g
        g1.setXYPlayer1(pre_x, pre_y)
        # print("X = " + str(pre_x) + " Y = " + str(pre_y))
        # g1.setXYPlayer_Oppenent(x_next_opponent, y_next_opponent)

        return IntPair(x_next, y_next)


In [10]:
p1 = MinimaxPlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

MinimaxPlayer.g = Game(p1, p2)

number_of_matches = 1
score1, score2 = MinimaxPlayer.g.start(number_of_matches)

print(score1 / number_of_matches)


Real X = 0 Real Y = 0
Real X = 1 Real Y = 0
Real X = 1 Real Y = 0
Real X = 2 Real Y = 1
Real X = 2 Real Y = 1
Real X = 3 Real Y = 2
Real X = 3 Real Y = 2
Real X = 4 Real Y = 3
Real X = 4 Real Y = 3
Real X = 5 Real Y = 3
Real X = 5 Real Y = 3
Real X = 5 Real Y = 4
Real X = 5 Real Y = 4
Real X = 5 Real Y = 5
Real X = 5 Real Y = 5
Real X = 4 Real Y = 6
Real X = 4 Real Y = 6
Real X = 5 Real Y = 6
Real X = 5 Real Y = 6
Real X = 6 Real Y = 6
Real X = 6 Real Y = 6
Real X = 7 Real Y = 7
Real X = 7 Real Y = 7
Real X = 6 Real Y = 7
Player 2 has exceeded the time limit
Player 1 has won

1.0


In [15]:
# My Test
global_score_1 = 0
global_score_2 = 0
for i in range(10):
    p1 = MinimaxPlayer(1, 0, 0)
    p2 = NaivePlayer(2, 7, 7)
    # p2 = MinimaxPlayer(2, 7, 7)

    MinimaxPlayer.g = Game(p1, p2)

    number_of_matches = 1
    score1, score2 = MinimaxPlayer.g.start(number_of_matches)

    print(score1 / number_of_matches)
    global_score_1 += score1
    global_score_2 += score2
    
print("Score 1 is " + str(global_score_1))
print("Score 2 is " + str(global_score_2))

Real X = 0 Real Y = 0
Real X = 0 Real Y = 1
Real X = 0 Real Y = 1
Real X = 1 Real Y = 2
Real X = 1 Real Y = 2
Real X = 2 Real Y = 3
Real X = 2 Real Y = 3
Real X = 3 Real Y = 4
Real X = 3 Real Y = 4
Real X = 4 Real Y = 5
Real X = 4 Real Y = 5
Real X = 5 Real Y = 5
Real X = 5 Real Y = 5
Real X = 6 Real Y = 5
Real X = 6 Real Y = 5
Real X = 6 Real Y = 6
Real X = 6 Real Y = 6
Real X = 7 Real Y = 6
Player 2 has exceeded the time limit
Player 1 has won

1.0
Real X = 0 Real Y = 0
Real X = 1 Real Y = 0
Real X = 1 Real Y = 0
Real X = 0 Real Y = 1
Real X = 0 Real Y = 1
Real X = 1 Real Y = 2
Real X = 1 Real Y = 2
Real X = 2 Real Y = 3
Real X = 2 Real Y = 3
Real X = 3 Real Y = 4
Real X = 3 Real Y = 4
Real X = 4 Real Y = 5
Real X = 4 Real Y = 5
Real X = 5 Real Y = 5
Real X = 5 Real Y = 5
Real X = 6 Real Y = 5
Real X = 6 Real Y = 5
Real X = 7 Real Y = 6
Real X = 7 Real Y = 6
Real X = 6 Real Y = 6
Real X = 6 Real Y = 6
Real X = 6 Real Y = 7
Player 2 has exceeded the time limit
Player 1 has won

1.0
Re

In [14]:
# My Test
global_score_1 = 0
global_score_2 = 0
for i in range(4):
    p1 = MinimaxPlayer(1, 0, 0)
    p2 = NaivePlayer(2, 7, 7)

    MinimaxPlayer.g = Game(p1, p2)

    number_of_matches = 1
    score1, score2 = MinimaxPlayer.g.start(number_of_matches)

    print(score1 / number_of_matches)
    global_score_1 += score1
    global_score_2 += score2
    
print("Score 1 is " + str(global_score_1))
print("Score 2 is " + str(global_score_2))

Real X = 0 Real Y = 0
Real X = 1 Real Y = 1
Real X = 1 Real Y = 1
Real X = 2 Real Y = 2
Real X = 2 Real Y = 2
Real X = 3 Real Y = 3
Real X = 3 Real Y = 3
Real X = 4 Real Y = 4
Real X = 4 Real Y = 4
Real X = 4 Real Y = 5
Real X = 4 Real Y = 5
Real X = 5 Real Y = 4
Real X = 5 Real Y = 4
Real X = 5 Real Y = 3
Real X = 5 Real Y = 3
Real X = 6 Real Y = 3
Real X = 6 Real Y = 3
Real X = 6 Real Y = 4
Real X = 6 Real Y = 4
Real X = 7 Real Y = 3
Player 2 has exceeded the time limit
Player 1 has won

1.0
Real X = 0 Real Y = 0
Real X = 1 Real Y = 0
Real X = 1 Real Y = 0
Real X = 2 Real Y = 1
Real X = 2 Real Y = 1
Real X = 3 Real Y = 1
Real X = 3 Real Y = 1
Real X = 4 Real Y = 2
Real X = 4 Real Y = 2
Real X = 5 Real Y = 3
Real X = 5 Real Y = 3
Real X = 5 Real Y = 4
Real X = 5 Real Y = 4
Real X = 5 Real Y = 5
Real X = 5 Real Y = 5
Real X = 6 Real Y = 6
Real X = 6 Real Y = 6
Real X = 6 Real Y = 7
Real X = 6 Real Y = 7
Real X = 5 Real Y = 7
Real X = 5 Real Y = 7
Real X = 4 Real Y = 7
Real X = 4 Real Y